In [17]:
import sqlalchemy
import geopandas as gp
import pandas as pd
from shapely.geometry import Point
%load_ext autotime
import h3
import numpy as np

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 682 µs (started: 2022-02-14 20:17:46 -05:00)


# TRY: ogr2ogr download postgis -> geojson (should be faster[?] than geopandas.from_postgis)
- ALSO: foreign table queries (alarms)

In [ ]:
# TODO HERE NOW

In [ ]:
# TODO HERE NOW

# Putting together

In [2]:
pg_driver = 'postgresql+psycopg2'
pg_user = 'boss_user'
pg_pass = 'passDEV9g47uibjn2ijovZNEW' # boss dev
pg_host = 'dev-boss.db.ready.net' # boss dev
pg_port = '5432'
pg_db = 'boss_db_dev'

# # SRID = 4326 used by GPS system
CRS = 4326

engine = sqlalchemy.create_engine(f"{pg_driver}://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}") 

cols = [
    'u.id',
    # 'u.map_point',
    'u.first_name',
    'u.last_name',
    'u.account_status',
    'u.account_type', 
    'u.service_status',
    'op.organization_id',
    'op.role',
    'a.lat',
    'a.lon']
select_cols = ','.join(cols)

# left outer join = include users not found in ogr_permission table
# inner join = only include users found in ogr_permission table
sql1 = f"""
    select {select_cols}
    from users u
    inner join organization_permissions op on (u.id = op.user_id)
    left outer join subscriptions s on u.id = s.user_id
    inner join addresses a on s.service_address_id = a.id 
    where u.account_status = 'subscriber'
    """

sql2 = f"""
    select {select_cols}
    from users u 
    inner join organization_permissions op on (u.id = op.user_id)
    inner join addresses a on u.mailing_address_id = a.id
    where u.account_status != 'subscriber'
    """


time: 21.1 ms (started: 2022-02-14 20:01:42 -05:00)


In [3]:
# 10sec now!
udf1 = pd.read_sql_query(sql1, engine)
udf2 = pd.read_sql_query(sql2, engine)
udf1.shape, udf2.shape, len(set(udf1.id).intersection(set(udf2.id)))

((63793, 10), (473573, 10), 0)

time: 9.49 s (started: 2022-02-14 20:01:49 -05:00)


In [ ]:
# # 30sec: geopandas SLOW
# udf1 = gp.GeoDataFrame.from_postgis(sql1, engine, geom_col='map_point', crs=CRS) 
# udf2 = gp.GeoDataFrame.from_postgis(sql2, engine, geom_col='map_point', crs=CRS) 


In [105]:
# # Combined query: generate more rows, but so hard to decipher!

# cols = [
#     'u.id',
#     'u.map_point',
#     'u.first_name',
#     'u.last_name',
#     'u.account_status',
#     'u.account_type', 
#     'u.service_status',
#     'op.organization_id',
#     'op.role']

# select_cols = ','.join(cols)

# select_sql = f"""select {select_cols},
# case
# when u.account_status = 'subscriber' then a1.lat
#                                      else a2.lat
# end as lat,
# case
# when u.account_status = 'subscriber' then a1.lon
#                                      else a2.lon
# end as lon
# from    users u
# inner join organization_permissions op on (u.id = op.user_id)
# left join subscriptions s  on (u.id = s.user_id)
# left join addresses     a1 on (s.service_address_id = a1.id)
# left join addresses     a2 on (u.mailing_address_id = a2.id)"""

# udf_test = gp.GeoDataFrame.from_postgis(select_sql, engine, geom_col='map_point', crs=4326) 

# len(set(udf.id)-set(udf_test.id)), len(set(udf_test.id)-set(udf.id))
# # (0, 17675)

time: 27.6 s (started: 2022-02-10 18:35:24 -05:00)


In [4]:
# combine 
udf = pd.concat([udf1, udf2], ignore_index=True)
### NOTE: id i.e. user_id should not be used as feature id (i.e. id for distinct points) because there are rows with same user_id (due to earlier JOIN statements)
udf = udf.rename(columns={'id': 'user_id'})

# drop rows with empty lat,lon [for now]
udf = udf[udf.lat.notnull() | udf.lon.notnull()]

# udf.lon = round(udf.lon,5)
# udf.lat = round(udf.lat,5)
udf.shape

(537363, 10)

time: 548 ms (started: 2022-02-14 20:02:03 -05:00)


In [425]:
# and tippecanoe doesn't need map_point col to mbtile
# udf = udf.drop(columns='map_point')
# [for now] overwrite all map_point using lat, lon 
### later: iff map_point are all correct, and populated ; then drop missing map_point if any

# mask = (udf['map_point'].isnull())
# udf.loc[mask, 'map_point'] = udf.loc[mask, ['lon', 'lat']].apply(lambda row: Point(row.lon, row.lat), axis=1)

# # APLLY is always slow 24sec
# udf['map_point'] = udf.apply(lambda row: Point(row.lon, row.lat), axis=1)

# # 17 sec
# udf['map_point'] = [Point(xy) for xy in zip(udf.lon, udf.lat)]


# still slow: 13 sec
# udf['map_point'] = gp.points_from_xy(udf.lon, udf.lat, crs= CRS)

time: 93 ms (started: 2022-02-11 12:57:17 -05:00)


# [for now] drop role=admin rows,
### remaining users are all subscribers, so drop column role as well

In [5]:
udf.role.value_counts()

subscriber    537344
admin              6
Name: role, dtype: int64

time: 49.4 ms (started: 2022-02-14 20:02:24 -05:00)


In [466]:
udf = udf[udf.role != 'admin']
udf = udf.drop(columns='role')


time: 310 ms (started: 2022-02-11 14:40:03 -05:00)


# drop service_status [for now]; and DONT NEED lat, lon anymore

In [467]:
udf = udf.drop(columns=['service_status'])

time: 90.7 ms (started: 2022-02-11 14:41:37 -05:00)


# fill na for account_type and account_status

In [468]:
udf.fillna({'account_type': 'null_acc_type', 'account_status': 'null_acc_status'}, inplace=True)

time: 143 ms (started: 2022-02-11 14:41:39 -05:00)


# Create needed *-count columns for cluster views

In [469]:
account_statuses = ['occupants',
'suspended',
'opportunity',
'lead',
'cancelled',
'subscriber',
'prospect',
'ineligible',
'pending',
'null_acc_status']

account_types = ['MDU',
'commercial',
'residential',
'null_acc_type']

time: 381 µs (started: 2022-02-11 14:41:43 -05:00)


In [405]:
# # 3 min :: SUPER SLOW!
# def cat_acc_status(r):
#     d = {}
#     # all new cols have count =0, except the corresponding account status
#     for acc_stt_cnt in account_statuses_counts:
#         d[acc_stt_cnt] = 0
#     d[f"{r['account_status']}_count"] = 1
#     return pd.Series(d, index = account_statuses_counts)
# udf_test = udf.apply(cat_acc_status, axis=1)
# set(udf_test.sum(axis=1))

time: 276 µs (started: 2022-02-10 22:14:31 -05:00)


### SUPER FAST!

In [454]:
udf

,user_id,first_name,last_name,account_status,account_type,organization_id,lat,lon,occupants_count,suspended_count,...,cancelled_count,subscriber_count,prospect_count,ineligible_count,pending_count,null_acc_status_count,MDU_count,commercial_count,residential_count,null_acc_type_count
0,3449,Lance,Nichols,subscriber,residential,206,37.087292,-94.508629,0,0,...,0,1,0,0,0,0,0,0,1,0
1,3453,Angela,Williams,subscriber,residential,206,37.018265,-94.737060,0,0,...,0,1,0,0,0,0,0,0,1,0
2,3454,Terry and Jackie,Osborne,subscriber,residential,206,36.911465,-94.426280,0,0,...,0,1,0,0,0,0,0,0,1,0
3,448775,Jase,Wilson,subscriber,residential,77,43.603029,-116.405121,0,0,...,0,1,0,0,0,0,0,0,1,0
4,3455,David,McLean,subscriber,residential,206,37.138456,-94.565377,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913037,386571,Minnie,Kovacek,lead,residential,77,39.745690,-94.833118,0,0,...,0,0,0,0,0,0,0,0,1,0
913038,386551,Zoie,Jenkins,lead,residential,77,39.745616,-94.825848,0,0,...,0,0,0,0,0,0,0,0,1,0
913039,386098,Luisa,Pacocha,lead,residential,77,39.756180,-94.812600,0,0,...,0,0,0,0,0,0,0,0,1,0
913040,386035,Taya,Stanton,lead,residential,77,39.758440,-94.801341,0,0,...,0,0,0,0,0,0,0,0,1,0


time: 576 ms (started: 2022-02-11 13:09:46 -05:00)


In [455]:
for acc_stt in account_statuses:
    udf[f"{acc_stt}_count"] = np.where(udf.account_status == acc_stt, 1, 0)
for acc_tp in account_types:
    udf[f"{acc_tp}_count"] = np.where(udf.account_type == acc_tp, 1, 0)    

# sanity check: sum of newly created *-count cols should be 2 for every row
set(udf.iloc[:,8:].sum(axis=1))

{2}

time: 1.12 s (started: 2022-02-11 13:10:01 -05:00)


In [456]:
udf.isnull().sum()

user_id                     0
first_name               8163
last_name                   0
account_status              0
account_type                0
organization_id             0
lat                         0
lon                         0
occupants_count             0
suspended_count             0
opportunity_count           0
lead_count                  0
cancelled_count             0
subscriber_count            0
prospect_count              0
ineligible_count            0
pending_count               0
null_acc_status_count       0
MDU_count                   0
commercial_count            0
residential_count           0
null_acc_type_count         0
dtype: int64

time: 260 ms (started: 2022-02-11 13:10:06 -05:00)


# output different geojson for different point/cluster views

In [ ]:
# top 5 are: 
444 UF demo, 
451 highline, 
77 FATFiber
432 UF
214 Wisper ISP

In [411]:
udf.organization_id.value_counts().head(7)

444    259881
451    228814
77      77995
432     65910
214     16192
206      4297
216      1001
Name: organization_id, dtype: int64

time: 5.49 ms (started: 2022-02-10 22:15:49 -05:00)


## save to UF-demo ogr for now

In [459]:
uf_demo = udf.loc[udf.organization_id == 444].drop(columns=['organization_id'])

time: 2.96 s (started: 2022-02-11 13:13:20 -05:00)


In [470]:
uf_demo.sample(5)

,user_id,first_name,last_name,account_status,account_type,lat,lon,occupants_count,suspended_count,opportunity_count,...,cancelled_count,subscriber_count,prospect_count,ineligible_count,pending_count,null_acc_status_count,MDU_count,commercial_count,residential_count,null_acc_type_count
639361,2411213,Vaughan Richard M & Cheryl J,,occupants,residential,39.386724,-94.578468,1,0,0,...,0,0,0,0,0,0,0,0,1,0
816586,2423622,Kcbs Llc,,occupants,MDU,39.209346,-94.689246,1,0,0,...,0,0,0,0,0,0,1,0,0,0
384367,1754336,Audra,Edwards,lead,commercial,39.336711,-94.265846,0,0,0,...,0,0,0,0,0,0,0,1,0,0
763841,2428392,Brighton Crossing 2 Llc,,occupants,MDU,39.238495,-94.513270,1,0,0,...,0,0,0,0,0,0,1,0,0,0
448048,2388373,Hines Mary M,,occupants,residential,39.242266,-94.587211,1,0,0,...,0,0,0,0,0,0,0,0,1,0


time: 21.4 ms (started: 2022-02-11 16:37:01 -05:00)


In [461]:
# tippecanoe read lat/lon cols from csv file, else have to save as geojson using geopandas (slow)
uf_demo.to_csv('data/ufdemo.csv', index=False)

time: 2.01 s (started: 2022-02-11 13:36:28 -05:00)


In [462]:
# uf_demo.to_json('data/ufdemo.json', orient='records')

time: 2.74 s (started: 2022-02-11 13:36:46 -05:00)


In [ ]:
## TODO : mbtiling and visualize different layers

# mbtiling # 1min

In [ ]:
# commands
# To see layer name: use CLI: 
# ogrinfo users-table/data/users.json 
# = 1: users (Point) = (users layer, geom type = Point)

ogrinfo -al ~/users-table/data/users.json | head -6
# Layer name: users
# Geometry: Point
# Feature Count: 675577

ogrinfo -al users-table/data/users.json >> users-table/ogrinfo.out

$ bash mbtiling.bash

# NOTE: mbtiles output is saved at ~/update-regrid/data/users.mbtiles
# and would be serve by tileserver-gl at port 5000 (same docker service/location where parcel mbtiles are served)

# update config.json at ~/update-regrid/data/config.json
cat > ~/update-regrid/data/config.json


In [ ]:
{
  "options": {
    "paths": { "mbtiles": "/data" }
  },
  "styles": {

  },
  "data": {
       "parcels_ak" : { "mbtiles": "parcels_ak.mbtiles" }, 
       "parcels_al" : { "mbtiles": "parcels_al.mbtiles" }, 
       "parcels_ar" : { "mbtiles": "parcels_ar.mbtiles" }, 
       "parcels_az" : { "mbtiles": "parcels_az.mbtiles" }, 
       "parcels_ca" : { "mbtiles": "parcels_ca.mbtiles" }, 
       "parcels_co" : { "mbtiles": "parcels_co.mbtiles" }, 
       "parcels_ct" : { "mbtiles": "parcels_ct.mbtiles" }, 
       "parcels_dc" : { "mbtiles": "parcels_dc.mbtiles" }, 
       "parcels_de" : { "mbtiles": "parcels_de.mbtiles" }, 
       "parcels_fl" : { "mbtiles": "parcels_fl.mbtiles" }, 
       "parcels_ga" : { "mbtiles": "parcels_ga.mbtiles" }, 
       "parcels_hi" : { "mbtiles": "parcels_hi.mbtiles" }, 
       "parcels_ia" : { "mbtiles": "parcels_ia.mbtiles" }, 
       "parcels_id" : { "mbtiles": "parcels_id.mbtiles" }, 
       "parcels_il" : { "mbtiles": "parcels_il.mbtiles" }, 
       "parcels_in" : { "mbtiles": "parcels_in.mbtiles" }, 
       "parcels_ks" : { "mbtiles": "parcels_ks.mbtiles" }, 
       "parcels_ky" : { "mbtiles": "parcels_ky.mbtiles" }, 
       "parcels_la" : { "mbtiles": "parcels_la.mbtiles" }, 
       "parcels_ma" : { "mbtiles": "parcels_ma.mbtiles" }, 
       "parcels_md" : { "mbtiles": "parcels_md.mbtiles" }, 
       "parcels_me" : { "mbtiles": "parcels_me.mbtiles" }, 
       "parcels_mi" : { "mbtiles": "parcels_mi.mbtiles" }, 
       "parcels_mn" : { "mbtiles": "parcels_mn.mbtiles" }, 
       "parcels_mo" : { "mbtiles": "parcels_mo.mbtiles" }, 
       "parcels_ms" : { "mbtiles": "parcels_ms.mbtiles" }, 
       "parcels_mt" : { "mbtiles": "parcels_mt.mbtiles" }, 
       "parcels_nc" : { "mbtiles": "parcels_nc.mbtiles" }, 
       "parcels_nd" : { "mbtiles": "parcels_nd.mbtiles" }, 
       "parcels_ne" : { "mbtiles": "parcels_ne.mbtiles" }, 
       "parcels_nh" : { "mbtiles": "parcels_nh.mbtiles" }, 
       "parcels_nj" : { "mbtiles": "parcels_nj.mbtiles" }, 
       "parcels_nm" : { "mbtiles": "parcels_nm.mbtiles" }, 
       "parcels_nv" : { "mbtiles": "parcels_nv.mbtiles" }, 
       "parcels_ny" : { "mbtiles": "parcels_ny.mbtiles" }, 
       "parcels_oh" : { "mbtiles": "parcels_oh.mbtiles" }, 
       "parcels_ok" : { "mbtiles": "parcels_ok.mbtiles" }, 
       "parcels_or" : { "mbtiles": "parcels_or.mbtiles" }, 
       "parcels_pa" : { "mbtiles": "parcels_pa.mbtiles" }, 
       "parcels_pr" : { "mbtiles": "parcels_pr.mbtiles" }, 
       "parcels_ri" : { "mbtiles": "parcels_ri.mbtiles" }, 
       "parcels_sc" : { "mbtiles": "parcels_sc.mbtiles" }, 
       "parcels_sd" : { "mbtiles": "parcels_sd.mbtiles" }, 
       "parcels_tn" : { "mbtiles": "parcels_tn.mbtiles" }, 
       "parcels_tx" : { "mbtiles": "parcels_tx.mbtiles" }, 
       "parcels_ut" : { "mbtiles": "parcels_ut.mbtiles" }, 
       "parcels_va" : { "mbtiles": "parcels_va.mbtiles" }, 
       "parcels_vt" : { "mbtiles": "parcels_vt.mbtiles" }, 
       "parcels_wa" : { "mbtiles": "parcels_wa.mbtiles" }, 
       "parcels_wi" : { "mbtiles": "parcels_wi.mbtiles" }, 
       "parcels_wv" : { "mbtiles": "parcels_wv.mbtiles" }, 
       "parcels_wy" : { "mbtiles": "parcels_wy.mbtiles" },
       "users" : { "mbtiles": "users.mbtiles" }
  }
}

# restart tileserver 

In [ ]:
docker restart tileserver
